In [1]:
!pip install sentence_transformers

트랜스포머를 이용한 한국어 챗봇 실습에서 사용했던 데이터를 그대로 사용합니다.

In [2]:
import urllib.request
import pandas as pd

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


문장 임베딩을 얻기 위해서 사전 훈련된 BERT를 로드합니다. 여기서는 한국어도 포함되어 학습된 다국어 모델을 로드합니다.

In [5]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

모델의 이름은 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'인데 이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미입니다. 다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.

SentenceTransformer로 로드할 수 있는 다양한 모델에 대한 리스트는 아래의 링크에서 확인 가능합니다.
해당 링크에 한국어 버전의 모델들 또한 공개되어 있으니 방문해보세요.

링크 : https://huggingface.co/models?library=sentence-transformers

데이터에서 모든 질문열. 즉, train_data['Q']에 대해서 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장합니다.

In [29]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")
encoding = tokenizer("", return_tensors='tf')
print(encoding)
for data in encoding['input_ids'][0].numpy():
    print(tokenizer.decode(data))
    
fisrt = model.encode("")
second = model.encode("12")
third = model.encode("시")
print(fisrt)
print(second)
print(third)

aver = model.encode("12시")
print(aver)

{'input_ids': <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[0, 2]])>, 'attention_mask': <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[1, 1]])>}
<s>
</s>
[ 1.39567852e-01 -1.72085017e-02  1.96638179e+00 -1.08046867e-01
  2.58146346e-01  4.59555328e-01 -8.38613510e-02  2.86948293e-01
 -1.27826825e-01 -1.89199552e-01 -2.35529557e-01  1.65173560e-01
 -1.92231327e-01  4.49706852e-01  9.45668042e-01 -2.05366716e-01
 -6.32603467e-01  2.20395312e-01  3.73688698e-01 -5.87851286e-01
  5.87897226e-02  4.57822323e-01 -9.44230333e-02 -5.01142681e-01
 -1.68331921e-01 -7.08800197e-01  3.24306995e-01 -1.26857126e+00
  2.32869118e-01 -5.04183695e-02  3.00817698e-01 -3.45330983e-01
  4.63951170e-01  3.66240323e-01 -8.36096257e-02 -1.46067202e-01
 -3.61397803e-01  2.68724322e-01 -1.67842835e-01 -1.30535692e-01
  9.86966372e-01  2.83194222e-02  7.52260208e-01  8.19057971e-02
 -1.77749187e-01 -2.56059263e-02 -2.93025345e-01  2.38658547e-01
 -5.74701190e-01 -5.78889847e-01 -1.01185715e+00 -5.9

In [6]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [10]:
train_data['embedding'][0].shape

(768,)

In [30]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [31]:
A = np.array([2,3])
print(norm(A))
B = np.array([3,1])
print(norm(B))
print(dot(A, B)/(norm(A)*norm(B)))

3.605551275463989
3.1622776601683795
0.7893522173763263


In [32]:
A = np.array([3,0])
print(norm(A))
B = np.array([0,3])
print(norm(B))
print(dot(A, B)/(norm(A)*norm(B)))

3.0
3.0
0.0


In [33]:
A = np.array([-3,0])
print(norm(A))
B = np.array([3,0])
print(norm(B))
print(dot(A, B)/(norm(A)*norm(B)))

3.0
3.0
-1.0


In [35]:
A = np.array([1,1,1,1,0,3])
print(norm(A))
B = np.array([2,2,2,2,0,2])
print(norm(B))
print(dot(A, B)/(norm(A)*norm(B)))

3.605551275463989
4.47213595499958
0.8682431421244593


두 개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim를 정의합니다.

In [36]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [37]:
embedding = model.encode('결혼하고싶어')
print(embedding.shape)

(768,)


return_answer 함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열. 즉, train_data['embedding']에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냅니다. 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴합니다.

In [38]:
def return_similar_answer(input):
    embedding = model.encode(input)
    
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

이제 챗봇을 테스트해봅시다.

In [39]:
return_similar_answer('결혼하고싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [40]:
return_similar_answer('나랑 커피먹을래?')

'카페인이 필요한 시간인가 봐요.'

In [41]:
return_similar_answer('반가워')

'저도 반가워요.'

In [42]:
return_similar_answer('사랑해')

'상대방에게 전해보세요.'

In [43]:
return_similar_answer('너는 누구니?')

'저는 위로봇입니다.'

In [44]:
return_similar_answer('영화')

'저도 영화 보여주세요.'

In [45]:
return_similar_answer('너무 짜증나')

'짜증날 땐 짜장면'

In [46]:
return_similar_answer('화가납니다')

'화를 참는 연습을 해보세요.'

In [47]:
return_similar_answer('나랑 놀자')

'지금 그러고 있어요.'

In [48]:
return_similar_answer('나랑 게임하자')

'같이 놀아요.'

In [49]:
return_similar_answer('출근하기싫어')

'씻고 푹 쉬세요.'

In [50]:
return_similar_answer('여행가고싶다')

'이김에 떠나보세요.'

In [51]:
return_similar_answer('너 말 잘한다')

'그런 사람이 있으면 저 좀 소개시켜주세요.'